## Download dataset

In [ ]:
import os
import urllib.request
import zipfile
import shutil

url = "https://3dshapenets.cs.princeton.edu/ModelNet10.zip"
zip_path = "ModelNet10.zip"
dataset_folder = "ModelNet10"

if not os.path.exists(dataset_folder):
    urllib.request.urlretrieve(url, zip_path)
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall()

    # Remove the zip file to save space
    os.remove(zip_path)

    #Created when extracting. Not useful to windows users.
    if os.path.exists("__MACOSX"):
        shutil.rmtree("__MACOSX")  


## For windows users

In [ ]:
import os

dataset_directory = os.path.join(os.getcwd(), "ModelNet10")

# Walk through the directory and remove .DS_Store files
for root, dirs, files in os.walk(dataset_directory):
    for file in files:
        if file == '.DS_Store':
            os.remove(os.path.join(root, file))

print("Removed all .DS_Store files from the dataset.")

## Precompute voxelization

In [ ]:
import os
import numpy as np
import trimesh

In [ ]:
# Pads to have consistent size. 
# Centers to make training better, network dont have to learn positions, only shape.
def voxelize(mesh, res):
    pitch = mesh.bounding_box.extents.max() / res
    voxel_grid = mesh.voxelized(pitch=pitch)
    voxel_matrix = voxel_grid.matrix.astype(np.float32)

    padded = np.zeros((res, res, res), dtype=np.float32)
    shape = voxel_matrix.shape
    min_dim = np.minimum(shape, (res, res, res))

    # Find which index the object should start, if the object is (30,30,30), and res = 32, 
    # the object should start at 1 (1 padding to the left and 1 to the right).
    start = [(res - min_dim[i]) // 2 for i in range(3)]
    end = [start[i] + min_dim[i] for i in range(3)]
    padded[start[0]:end[0], start[1]:end[1], start[2]:end[2]] = voxel_matrix[:min_dim[0], :min_dim[1], :min_dim[2]]

    return padded

In [ ]:
dataset_directory = "../ModelNet10"
output_directory = "./ModelNet10_voxelized_64"
res = 32

if not os.path.exists(output_directory):
    for object_folder in os.listdir(dataset_directory):
        if object_folder == "README.txt":
            continue

        object_folder_path = os.path.join(dataset_directory, object_folder)

        for split in ['train', 'test']:
            split_path = os.path.join(object_folder_path, split)

            save_split_path = os.path.join(output_directory, object_folder, split)
            os.makedirs(save_split_path, exist_ok=True)

            for file in os.listdir(split_path):
                if file.endswith('.off'):
                    file_path = os.path.join(split_path,file)

                    mesh = trimesh.load_mesh(file_path)
                    voxels = voxelize(mesh, res)

                    save_path = os.path.join(save_split_path, file.replace('.off','.npy'))
                    np.save(save_path, voxels)